# Genie Top 100

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
url='https://www.genie.co.kr/chart/top200'
req=requests.get(url)
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
req = requests.get(url, headers=header)

In [7]:
# 2,3,4 페이지
'''
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=1
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=2
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=3
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=4
'''

'\nhttps://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=1\nhttps://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=2\nhttps://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=3\nhttps://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=4\n'

In [9]:
base_url='https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg='
url=base_url + str(1)
req = requests.get(url, headers=header)
soup=BeautifulSoup(req.text,'html.parser')

### 1. 원하는 목록 찾기

In [10]:
trs=soup.select('tr.list') #tr 태그의 클래스 명이 list
len(trs)

50

In [13]:
# 개발자 도구
trs=soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

## 2. 하나의 데이터를 선택해서 원하는 정보 추출하기

In [14]:
tr=trs[0]

In [15]:
#rank
rank=tr.select_one('.number').get_text()
rank

'1\n                                        \n                                    \n유지\n\n'

In [16]:
rank.split('\n')[0]

'1'

In [17]:
rank=int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [18]:
# 제목
title=tr.select_one('.title.ellipsis').get_text().strip()
title

'그때 그 순간 그대로 (그그그)'

In [19]:
# 가수
artist=tr.select_one('.artist.ellipsis').get_text().strip()
artist

'WSG워너비 (가야G)'

In [21]:
#앨범
album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'WSG워너비 1집'

### 3. 한페이지 내의 정보

In [22]:
lines=[]
for tr in trs:
  rank=int(tr.select_one('.number').get_text().split('\n')[0])
  title=tr.select_one('.title.ellipsis').get_text().strip()
  artist=tr.select_one('.artist.ellipsis').get_text().strip()
  album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
  lines.append([rank,title,artist,album])

4. 모든 페이지에 대해서 가져오기

In [23]:
lines=[]
for page in range(1,5):
  url=base_url + str(1)
  req = requests.get(url, headers=header)
  soup=BeautifulSoup(req.text,'html.parser')
  trs=soup.select('tr.list')
  for tr in trs:
    rank=int(tr.select_one('.number').get_text().split('\n')[0])
    title=tr.select_one('.title.ellipsis').get_text().strip()
    artist=tr.select_one('.artist.ellipsis').get_text().strip()
    album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank,title,artist,album])

5.데이터 프레임으로 만들고 저장하기

In [24]:
df=pd.DataFrame(lines,columns=['순위','제목','가수','앨범'])
df.head()

,순위,제목,가수,앨범
0,1,그때 그 순간 그대로 (그그그),WSG워너비 (가야G),WSG워너비 1집
1,2,Attention,NewJeans,NewJeans 1st EP 'New Jeans'
2,3,보고싶었어,WSG워너비 (4FIRE),WSG워너비 1집
3,4,LOVE DIVE,IVE (아이브),LOVE DIVE
4,5,SNEAKERS,ITZY (있지),CHECKMATE


In [25]:
df.head(95).tail()

,순위,제목,가수,앨범
90,41,That's Hilarious,Charlie Puth,That's Hilarious
91,42,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
92,43,FEARLESS,LE SSERAFIM (르세라핌),FEARLESS
93,44,해요 (2022),#안녕,해요 (2022)
94,45,ELEVEN,IVE (아이브),ELEVEN


6. 4과정의 버그를 잡을 때까지 수정

In [27]:
lines=[]
for page in range(1,5):
  url=base_url + str(1)
  req = requests.get(url, headers=header)
  soup=BeautifulSoup(req.text,'html.parser')
  trs=soup.select('tr.list')
  for index,tr in enumerate(trs):
    try:
      rank=int(tr.select_one('.number').get_text().split('\n')[0])
      title=tr.select_one('.title.ellipsis').string.strip()
      artist=tr.select_one('.artist.ellipsis').get_text().strip()
      album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
      lines.append([rank,title,artist,album])
    except:
      print(page,index)

In [28]:
tr=trs[27]
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                손이 참 곱던 그대'

In [29]:
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

'손이 참 곱던 그대'

In [31]:
lines=[]
for page in range(1,5):
  url=base_url + str(1)
  req = requests.get(url, headers=header)
  soup=BeautifulSoup(req.text,'html.parser')
  trs=soup.select('tr.list')
  for tr in trs:
    rank=int(tr.select_one('.number').get_text().split('\n')[0])
    try:
      title=tr.select_one('.title.ellipsis').string.strip()
    except:
      title=tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist=tr.select_one('.artist.ellipsis').get_text().strip()
    album=tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank,title,artist,album])

7.과정 실행

In [32]:
df=pd.DataFrame(lines,columns=['순위','제목','가수','앨범'])
df.head(94).tail()

,순위,제목,가수,앨범
89,40,취중고백,김민석 (멜로망스),취중고백
90,41,That's Hilarious,Charlie Puth,That's Hilarious
91,42,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
92,43,FEARLESS,LE SSERAFIM (르세라핌),FEARLESS
93,44,해요 (2022),#안녕,해요 (2022)


In [33]:
df.to_csv('지니Top200(220808).csv', index=False)